In [17]:
import torch, os, pickle, time, scipy
import torch_geometric as tg
from torch_geometric.data import Data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import os.path as osp
import networkx as nx
path='~/../../tigress/cj1223/merger_trees/isotrees/'
transform_path='~/../../scratch/gpfs/cj1223/GraphStorage/transformer'
all_cols=np.array([0,2,4,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,35]+list(range(37,60)))
targets=[8,11,14,15,16,17,18,23]

## load column names + other nice things

In [4]:
### targets
ex=f'../samout/{0}_{0}_{0}/galprop_0-99.dat'
pdc=pd.read_table(path+ex, skiprows=0, delimiter=',', nrows=41, header=None)

colst=pdc.iloc[targets,0]
cols_t = []
for i, col in enumerate(colst):
    cols_t.append(col[4:]+f'({i})') 
    
halos=pd.read_table(path+f'isotree_0_0_0.dat', skiprows=0, nrows=1, delimiter='\s+')
cols_h = []
for i, col in enumerate(halos.columns[all_cols]):
    if col[-1] == ')':
        cols_h.append(col[:-3]+f'({i})')
    else:
        cols_h.append(col+f'({i})')  
        
feat=['quantile', 'standard', 'power']

targ=['quant', 'stand']

In [5]:
case='vlarge_all_multit_z0.0'
transform=feat[0]
t=targ[0]

case=case+f'_{transform}'+f'_{t}'

datat=pickle.load(open(osp.expanduser(\
                f'~/../../../scratch/gpfs/cj1223/GraphStorage/{case}/data.pkl'), 'rb'))
print(case)

In [6]:
xs=[]
ys=[]
ls=[]
xf=[]
edge_index=[]
edge_attr=[]
for d in datat: #[:1000]
    xs.append(d.x.numpy())
    xf.append(d.x.numpy()[0])
    ys.append(d.y.numpy())
    edge_index.append(d.edge_index)
    edge_attr.append(d.edge_attr)
    l=len(d.x.numpy())
    ls.append(l)
xs=np.vstack(xs)
xf=np.vstack(xf)
ys=np.vstack(ys)
ls=np.array(ls)
split=np.cumsum(ls)

In [22]:
targ=[1]

a=input()
case='vlarge_all_smass_z0.0_quantile_quant'

dat=[]
for d in tqdm(datat):
    x = d.x
    y=d.y[targ]
    graph=Data(x=x, edge_index=d.edge_index, edge_attr=d.edge_attr, y=y)
    dat.append(graph)

print("Saving dataset")
if not osp.exists(osp.expanduser(f'~/../../../scratch/gpfs/cj1223/GraphStorage/{case}')):
    os.makedirs(osp.expanduser(f'~/../../../scratch/gpfs/cj1223/GraphStorage/{case}'))

with open(osp.expanduser(f'~/../../../scratch/gpfs/cj1223/GraphStorage/{case}/data.pkl'), 'wb') as handle:
    print(handle)
    pickle.dump(dat, handle)

a


100%|█████████████████████████████████████████████████████████████████| 108808/108808 [00:11<00:00, 9695.96it/s]


Saving dataset
<_io.BufferedWriter name='/home/cj1223/../../../scratch/gpfs/cj1223/GraphStorage/vlarge_all_smass_z0.0_quantile_quant/data.pkl'>


In [19]:
dat

[Data(x=[56, 43], edge_index=[2, 55], edge_attr=[55], y=[4]),
 Data(x=[250, 43], edge_index=[2, 249], edge_attr=[249], y=[4]),
 Data(x=[59, 43], edge_index=[2, 58], edge_attr=[58], y=[4]),
 Data(x=[49, 43], edge_index=[2, 48], edge_attr=[48], y=[4]),
 Data(x=[241, 43], edge_index=[2, 240], edge_attr=[240], y=[4]),
 Data(x=[261, 43], edge_index=[2, 260], edge_attr=[260], y=[4]),
 Data(x=[198, 43], edge_index=[2, 197], edge_attr=[197], y=[4]),
 Data(x=[136, 43], edge_index=[2, 135], edge_attr=[135], y=[4]),
 Data(x=[17332, 43], edge_index=[2, 17331], edge_attr=[17331], y=[4]),
 Data(x=[36, 43], edge_index=[2, 35], edge_attr=[35], y=[4]),
 Data(x=[43, 43], edge_index=[2, 42], edge_attr=[42], y=[4]),
 Data(x=[154, 43], edge_index=[2, 153], edge_attr=[153], y=[4]),
 Data(x=[95, 43], edge_index=[2, 94], edge_attr=[94], y=[4]),
 Data(x=[4310, 43], edge_index=[2, 4309], edge_attr=[4309], y=[4]),
 Data(x=[419, 43], edge_index=[2, 418], edge_attr=[418], y=[4]),
 Data(x=[24, 43], edge_index=[2, 2